In [6]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.cluster import MiniBatchKMeans
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin, BaseEstimator

In [7]:
from dateutil import parser

def get_nLast_dates(s, n = 1):
    date = parser.parse(s)
    index = dates.index(date)
    result = []
    for x in range(1, n + 1):
        former_index = index - x
        if (former_index >= 0):
            result.append(dates[former_index].strftime("%d/%m/%Y"))
        else: 
            result.append(None)
    return result

def create_columns_data(store, dept, current_date, column_count = 1, default_value = 0):
    dates = get_nLast_dates(current_date, column_count)
    result_dict = {}
    for date in dates:
        result_dict.update({'curr_date-' + str(dates.index(date) + 1): former_data.get((store, dept, date), default_value)})
    return result_dict
   

In [15]:
from datetime import datetime
feature_data_set = pd.io.parsers.read_csv('/Users/omer.h/Code/jupyter/Retail data/Features data set.csv')
sales_data_set = pd.io.parsers.read_csv('/Users/omer.h/Code/jupyter/Retail data/sales data-set.csv')
store_data_set = pd.io.parsers.read_csv('/Users/omer.h/Code/jupyter/Retail data/stores data-set.csv')

data_store_sales = pd.merge(sales_data_set, store_data_set, how='inner', on=['Store'], left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

data = pd.merge(data_store_sales, feature_data_set, how='inner', on=['Store', 'Date'], left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

In [17]:
former_Dataframe = data[['Store', 'Dept', 'Date', 'Weekly_Sales']]
former_data = former_Dataframe.groupby(['Store', 'Dept', 'Date']).mean().to_dict()['Weekly_Sales']
dates = sorted(list(map(lambda x:parser.parse(x), list(data['Date'].unique()))))

former_weekly_sales_raw = data.apply(lambda x: create_columns_data(x['Store'], x['Dept'], x['Date'], 3), axis=1)
former_weekly_sales = pd.DataFrame.from_records(former_weekly_sales_raw.values, index=data.index)
result = pd.concat([data, former_weekly_sales], axis=1)
data_backup = data
data = result
#data.Date = data.Date.apply(lambda x: parser.parse(x))
#data.Date = data_backup.Date

In [18]:
data.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday_x,Type,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday_y,curr_date-1,curr_date-2,curr_date-3
0,1,1,01/04/2011,20398.09,False,A,151315,59.17,3.524,NaN,NaN,NaN,NaN,NaN,214.837166,7.682,False,19124.58,55931.23,44912.86
1,1,2,01/04/2011,46991.58,False,A,151315,59.17,3.524,NaN,NaN,NaN,NaN,NaN,214.837166,7.682,False,35819.83,59889.32,49303.28
2,1,3,01/04/2011,8734.19,False,A,151315,59.17,3.524,NaN,NaN,NaN,NaN,NaN,214.837166,7.682,False,8266.14,12080.36,11986.84
3,1,4,01/04/2011,34451.90,False,A,151315,59.17,3.524,NaN,NaN,NaN,NaN,NaN,214.837166,7.682,False,36913.80,47893.23,39154.41
4,1,5,01/04/2011,23598.55,False,A,151315,59.17,3.524,NaN,NaN,NaN,NaN,NaN,214.837166,7.682,False,32280.82,85676.09,59589.48


In [10]:



y = data.Weekly_Sales
X = data.drop('Weekly_Sales', axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

train, test = train_test_split(data, test_size=0.2, random_state=42)

from auto_ml import Predictor
from auto_ml.utils import get_boston_dataset

df_train, df_test = data, data

column_descriptions = {
    'Weekly_Sales': 'output',
    'Store': 'categorical',
    'Dept': 'categorical',
     'Date': 'categorical',
    'isHoliday_x': 'categorical',
    'isHoliday_y': 'ignore',
}
print('asdasdasd')
#ml_predictor = Predictor(type_of_estimator='regressor', column_descriptions=column_descriptions)
                         #take,take_log_of_y = False, calibrate_final_model = True, feature_learning = True)

ml_predictor = Predictor(type_of_estimator='regressor', column_descriptions=column_descriptions)
ml_predictor.take_log_of_y = True
ml_predictor.calibrate_final_model = True
ml_predictor.feature_learning = True

ml_predictor.train(train, ml_for_analytics=True, model_names=['LinearRegression'])

y = test.Weekly_Sales
test = test.drop('Weekly_Sales', axis = 1)
ml_predictor.score(test, y)

#X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=0.2, random_state=42)


asdasdasd
Welcome to auto_ml! We're about to go through and make sense of your data using machine learning, and give you a production-ready pipeline to get predictions with.

If you have any issues, or new feature ideas, let us know at http://auto.ml
You are running on version 2.9.9
Now using the model training_params that you passed in:
{}
After overwriting our defaults with your values, here are the final params that will be used to initialize the model:
{'n_jobs': -2}
Running basic data cleaning
Performing feature scaling
Fitting DataFrameVectorizer
Now using the model training_params that you passed in:
{}
After overwriting our defaults with your values, here are the final params that will be used to initialize the model:
{'n_jobs': -2}


********************************************************************************************
About to fit the pipeline for the model LinearRegression to predict Weekly_Sales
Started at:
2018-02-28 17:08:58
Finished training the pipeline!
Total tra

/Users/omer.h/anaconda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


The printed list will only contain at most the top 100 features.
+----+-----------------+----------------+------------+-------------------+-------------------+-----------+-----------+-----------+-----------+
|    | Feature Name    |   Coefficients |      Delta |   FR_Decrementing |   FR_Incrementing |   FRD_abs |   FRI_abs |   FRD_MAD |   FRI_MAD |
|----+-----------------+----------------+------------+-------------------+-------------------+-----------+-----------+-----------+-----------|
| 50 | Date=25/05/2012 |      -115.8776 |   nan      |          nan      |          nan      |  nan      |  nan      |  nan      |  nan      |
| 72 | Date=28/10/2011 |       -73.5750 |   nan      |          nan      |          nan      |  nan      |  nan      |  nan      |  nan      |
| 76 | Date=04/03/2011 |       -60.5169 |   nan      |          nan      |          nan      |  nan      |  nan      |  nan      |  nan      |
|  3 | curr_date-2     |         0.2244 | 10334.9142 |        -2319.5048 |   

-9772.3547794004244

In [11]:
ml_predictor.save(file_name='auto_ml_saved_pipeline_GradientBoosting_withFormerFeatures1.dill')



We have saved the trained pipeline to a filed called "auto_ml_saved_pipeline_GradientBoosting_withFormerFeatures1.dill"
It is saved in the directory: 
/Users/omer.h/Code/jupyter/Retail data
To use it to get predictions, please follow the following flow (adjusting for your own uses as necessary:


`from auto_ml.utils_models import load_ml_model
`trained_ml_pipeline = load_ml_model("auto_ml_saved_pipeline_GradientBoosting_withFormerFeatures1.dill")
`trained_ml_pipeline.predict(data)`


Note that this pickle/dill file can only be loaded in an environment with the same modules installed, and running the same Python version.
This version of Python is:
sys.version_info(major=3, minor=6, micro=0, releaselevel='final', serial=0)


When passing in new data to get predictions on, columns that were not present (or were not found to be useful) in the training data will be silently ignored.
It is worthwhile to make sure that you feed in all the most useful data points though, to make sure you can

'/Users/omer.h/Code/jupyter/Retail data/auto_ml_saved_pipeline_GradientBoosting_withFormerFeatures1.dill'

In [12]:
data.head()


,Store,Dept,Date,Weekly_Sales,IsHoliday_x,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday_y
0,1,1,01/04/2011,20398.09,False,59.17,3.524,NaN,NaN,NaN,NaN,NaN,214.837166,7.682,False
1,1,2,01/04/2011,46991.58,False,59.17,3.524,NaN,NaN,NaN,NaN,NaN,214.837166,7.682,False
2,1,3,01/04/2011,8734.19,False,59.17,3.524,NaN,NaN,NaN,NaN,NaN,214.837166,7.682,False
3,1,4,01/04/2011,34451.90,False,59.17,3.524,NaN,NaN,NaN,NaN,NaN,214.837166,7.682,False
4,1,5,01/04/2011,23598.55,False,59.17,3.524,NaN,NaN,NaN,NaN,NaN,214.837166,7.682,False


False

In [6]:
file_name = ml_predictor.save()

{'Date': 'categorical',
 'Dept': 'categorical',
 'Store': 'categorical',
 'Weekly_Sales': 'output',
 'isHoliday_x': 'categorical',
 'isHoliday_y': 'ignore'}

In [38]:
file_name = ml_predictor.save()




We have saved the trained pipeline to a filed called "auto_ml_saved_pipeline.dill"
It is saved in the directory: 
/Users/omer.h/Code/jupyter/Retail data
To use it to get predictions, please follow the following flow (adjusting for your own uses as necessary:


`from auto_ml.utils_models import load_ml_model
`trained_ml_pipeline = load_ml_model("auto_ml_saved_pipeline.dill")
`trained_ml_pipeline.predict(data)`


Note that this pickle/dill file can only be loaded in an environment with the same modules installed, and running the same Python version.
This version of Python is:
sys.version_info(major=3, minor=6, micro=0, releaselevel='final', serial=0)


When passing in new data to get predictions on, columns that were not present (or were not found to be useful) in the training data will be silently ignored.
It is worthwhile to make sure that you feed in all the most useful data points though, to make sure you can get the highest quality predictions.


In [8]:
ml_predictor.ensemble_config

[]

In [31]:
ml_predictor.trained_final_model.training_features

['Temperature',
 'Fuel_Price',
 'MarkDown1',
 'MarkDown2',
 'MarkDown3',
 'MarkDown4',
 'MarkDown5',
 'CPI',
 'Unemployment',
 'IsHoliday_x',
 'IsHoliday_y',
 'Store=1',
 'Store=2',
 'Store=3',
 'Store=4',
 'Store=5',
 'Store=6',
 'Store=7',
 'Store=8',
 'Store=9',
 'Store=10',
 'Store=11',
 'Store=12',
 'Store=13',
 'Store=14',
 'Store=15',
 'Store=16',
 'Store=17',
 'Store=18',
 'Store=19',
 'Store=20',
 'Store=21',
 'Store=22',
 'Store=23',
 'Store=24',
 'Store=25',
 'Store=26',
 'Store=27',
 'Store=28',
 'Store=29',
 'Store=30',
 'Store=31',
 'Store=32',
 'Store=33',
 'Store=34',
 'Store=35',
 'Store=36',
 'Store=37',
 'Store=38',
 'Store=39',
 'Store=40',
 'Store=41',
 'Store=42',
 'Store=43',
 'Store=44',
 'Store=45',
 'Dept=1',
 'Dept=2',
 'Dept=3',
 'Dept=4',
 'Dept=5',
 'Dept=6',
 'Dept=7',
 'Dept=8',
 'Dept=9',
 'Dept=10',
 'Dept=11',
 'Dept=12',
 'Dept=13',
 'Dept=14',
 'Dept=16',
 'Dept=17',
 'Dept=18',
 'Dept=19',
 'Dept=20',
 'Dept=21',
 'Dept=22',
 'Dept=23',
 'Dept=24',

In [33]:

prediction = ml_predictor.predict(test)
p1 = pd.DataFrame(list(map(lambda x: round(x), prediction)))
p1 = p1.assign(predictions=y.values)
p1.columns = ['Predictions', 'Labels']
p1


,Predictions,Labels
0,2152,260.32
1,15856,20258.11
2,64717,74185.53
3,227,3476.29
4,15722,16081.85
5,20584,17170.40
6,37130,41520.03
7,40114,46721.54
8,17781,18705.07
9,797,1396.01


In [37]:
p2 = test.assign()

TypeError: assign() takes 1 positional argument but 2 were given

In [29]:
ml_predictor.model_names


['GradientBoostingRegressor']

In [17]:
train, test = train_test_split(data, test_size=0.2, random_state=42)

In [20]:
len(test)

84314

In [21]:
len(train)

337256